# Projeto Reclamações

Este projeto foi desenvolvido como parte das atividades do [Instituto Brasileiro de Defesa do Consumidor](https://idec.org.br/), que está analisando dados sobre reclamações registradas por consumidores em canais oficiais.

## Criando um banco de dados do Google Cloud Platform

Se o seu banco de dados já foi criado, siga para a próxima seção. Se não, continue nesta seção para obter as instruções para a criação do banco de dados.

Utilizaremos um banco de dados no Google Cloud Platform. Adotamos um banco de dados em nuvem porque, além de diminuir riscos, esse tipo de serviço permite que várias pessoas acessem e analisem os dados. O nosso banco de dados foi criado seguindo [este tutorial](https://drive.google.com/drive/u/1/folders/16egLadoDi9TdigfbeFEEvdxHaQUXfcYR) e as credenciais de acesso encontram-se [neste link](https://docs.google.com/document/d/11IWglGcNrUqUMTbU2jU3Ml_jOcO-vCW1SDk1ljen0ko/edit?usp=sharing).

Começaremos criando a tabela para registrar os dados de reclamações disponibilizados pela ANEEL. Para criar a tabela, foi utilizado o código SQL abaixo.

```sql
-- Criando tabela de reclamacoes da ANEEL
create table reclamacoes_aneel (
	distribuidora varchar(50),
	ano int,
	mes int,
	descricao varchar(150),
	reclamacoes_recebidas int,
	reclamacoes_procedentes int,
	prazo_medio_encerramento_horas real,
	prazo_medio_encerramento_dias real,
	reclamacoes_improcedentes int,
	reclamacoes_totais int
);
```

## Atualizando os dados

O banco de dados já foi previamente alimentado com os dados históricos, mas é importante realizar a sua atualização, uma vez que a ANEEL disponibiliza novos dados de reclamações todo mês (pelo menos em teoria) [neste link](https://www.aneel.gov.br/relatorios-de-servicos-comerciais).

Como você pode ver no link, a ANEEL não facilita a nossa vida disponibilizando um csv com os dados agregados. Em vez disso, eles disponibilizam tabelas que são geradas dinamicamente no site conforme você seleciona as opções. Assim, para armazenar esses dados e alimentar nosso banco, teríamos que copiar e colar em um csv. Como fazer isso manualmente seria impossível (sem condições!), vamos utilizar Python para fazer um web scraping (ou uma raspagem de dados) na página da ANEEL.

### Definindo os parâmetros e variáveis de entrada
Rode o código abaixo para definir os parâmetros e as variáveis de entrada que utilizaremos em nosso web scraping. Observe que você pode mudar o ano e mês de acordo com os dados que deseja obter.

In [11]:
url = 'https://www2.aneel.gov.br/aplicacoes_liferay/relatorios_de_qualidade_v2/'

# 1: RECLAMAÇÕES
# 2: QUALIDADE DO ATENDIMENTO TELEFÔNICO
# 3: QUALIDADE DO ATENDIMENTO COMERCIAL
# 4: IASC - ÍNDICE DE SATISFAÇÃO DO CONSUMIDOR
# 5: INADIMPLÊNCIA
# 6: TARIFA SOCIAL
# 7: UNIVERSALIZAÇÃO
tipo_relatorio = '1'

# 5_1_v2: Inadimplência por distribuidora por mês
# 5_2_v2: Inadimplência por distribuidora - evolução no ano
# 5_3_v2: Inadimplência e Suspensão de Fornecimento - Evolução por Classe por Distribuidora
# 5_4_v2: Inadimplência média e Suspensão de Fornecimento - Brasil
# 5_5_v2: Inadimplência média e Suspensão de Fornecimento por Classe - Brasil
relatorio = '1_7_v2'

# C: Concessionária
# P: Permissionária
# T: Todas
tipo_distribuidora = 'T'

# Substitua de acordo com os dados que deseja obter
anos_selecionados = [2021]
meses_selecionados = [1, 2]

dict_meses = {"1": "Janeiro",
              "2": "Fevereiro",
              "3": "Março",
              "4": "Abril",
              "5": "Maio",
              "6": "Junho",
              "7": "Julho",
              "8": "Agosto",
              "9": "Setembro",
              "10": "Outubro",
              "11": "Novembro",
              "12": "Dezembro"}

In [12]:
import pandas as pd

def clean_df_novos_dados():
    df_novos_dados = pd.DataFrame(columns=['distribuidora', 'ano', 'mes', 'descricao', 'reclamacoes_recebidas', 'reclamacoes_procedentes',
                                 'prazo_medio_encerramento_horas', 'prazo_medio_encerramento_dias', 'reclamacoes_improcedentes', 'reclamacoes_totais'])
    return df_novos_dados

clean_df_novos_dados()

,distribuidora,ano,mes,descricao,reclamacoes_recebidas,reclamacoes_procedentes,prazo_medio_encerramento_horas,prazo_medio_encerramento_dias,reclamacoes_improcedentes,reclamacoes_totais


### Fazendo requisição HTML para acessar a página da ANEEL
O código web scraping irá abrir uma janela do seu navegador e acessar a página da ANEEL. No nosso caso, utilizaremos o Chrome por ser o navegador mais popular. Para que o Python consiga acessar o seu navegador, você precisa ter o web driver do Chrome instalado. Se você não tiver o web driver instalado, você pode fazer o download da [neste link](https://chromedriver.chromium.org/downloads). Não esqueça de verificar qual a versão compatível com a sua versão do Chrome.

Depois de fazer o download, insira o caminho correspondente e rode o código abaixo. Ele define a função `requisicao_html()` que irá fazer uma requisição HTML para acessar a página da ANEEL. Se tudo der certo, você observará uma janela do Chrome sendo aberta e acessando a página da ANEEL (mágica!). Esse foi apenas um teste, então você pode fechar essa janela.

In [13]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
webdriver_path = r'C:\Users\Camila\Downloads\chromedriver_win32\chromedriver.exe'

def requisicao_html():
    # Faz requisição HTML para acessar o url desejado e retorna o objeto que será utilizado para interagir com a página
    chrome_options = Options()
    chrome_options.add_argument("--window-size=900,600")
    driver = webdriver.Chrome(webdriver_path,options=chrome_options)
    driver.get(url)
    return driver

requisicao_html()

<selenium.webdriver.chrome.webdriver.WebDriver (session="bf55cbec5569271c3173b57173a4e0b0")>

Se você navegou um pouco na página da ANEEL, você deve ter observado que é necessário selecionar uma série de parâmetros antes de chegar na tabela com os dados desejados (a ANEEL realmente não está facilitando o acesso à informação rs). O código a seguir define as funções que selecionam os parâmetros iniciais para nós.

Observe que estamos definindo a função `verifica_selecao()` para verificar se a opção que desejamos está realmente selecionada e a função `erro_stale_element()` para lidar com o erro de stale element. Esse é um erro comum em páginas geradas dinamicamente com JavaScript, como é o caso da nossa página da ANEEL. A função `erro_stale_element()` garante que o nosso código não terá problemas com esse tipo de erro.

In [14]:
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions

def seleciona_tipo_relatorio():
    try:
        select_relatorio_pai = Select(driver.find_element_by_xpath("//select[@id='select_tipo_relatorio_pai']"))
        select_relatorio_pai.select_by_value(tipo_relatorio)
        selecionou = verifica_selecao(select_relatorio_pai, tipo_relatorio)
        while selecionou != True:
            selecionou = verifica_selecao(select_relatorio_pai, tipo_relatorio)
        return selecionou
    except Exception as e:
        erro_stale_element('select_tipo_relatorio_pai')
        seleciona_tipo_relatorio()

def seleciona_relatorio():
    try:
        select_relatorio = Select(driver.find_element_by_xpath("//select[@id='select_tipo_relatorio']"))
        select_relatorio.select_by_value(relatorio)
        selecionou = verifica_selecao(select_relatorio, relatorio)
        while selecionou != True:
            selecionou = verifica_selecao(select_relatorio, relatorio)
        return selecionou
    except Exception as e:
        erro_stale_element('select_tipo_relatorio')
        seleciona_relatorio()

def seleciona_tipo_distribuidora():
    try:        
        select_tipo_distribuidora = Select(driver.find_element_by_xpath("//select[@id='select_classificacao_agente']"))
        select_tipo_distribuidora.select_by_value(tipo_distribuidora)
        selecionou = verifica_selecao(select_tipo_distribuidora, tipo_distribuidora)
        while selecionou != True:
            selecionou = verifica_selecao(select_tipo_distribuidora, tipo_distribuidora)
        return selecionou
    except Exception as e:
        erro_stale_element('select_classificacao_agente')
        seleciona_tipo_distribuidora()
        
def verifica_selecao(select, valor):
    opcao_selecionada = select.first_selected_option.get_attribute("value")
    if opcao_selecionada == valor:
        return True
    else:
        return False
        
def erro_stale_element(id_elemento):
    my_element_id = id_elemento
    ignored_exceptions = (NoSuchElementException,StaleElementReferenceException)
    your_element = WebDriverWait(driver, 10, ignored_exceptions=ignored_exceptions).until(expected_conditions.presence_of_element_located((By.ID, my_element_id)))

Utilize o código abaixo para chamar as funções que acabamos de definir e verificar se tudo está ocorrendo como esperado.

In [15]:
def primeiros_selects():
    if seleciona_tipo_relatorio():
        seleciona_relatorio()
    if seleciona_relatorio():
        seleciona_tipo_distribuidora()
    if seleciona_tipo_distribuidora():
        return True

import time
driver = requisicao_html()
time.sleep(1)
if primeiros_selects():
    print('Tudo certo!')

Tudo certo!


Após selecionar os primeiros parâmetros, a página da ANEEL gera dinamicamente um novo select, com uma lista de distirbuidoras. Vamos acessar essa lista para depois iterar sobre ela e obter os dados de reclamações de cada distribuidora.

A seguir, está definida a função `gera_lista_distribuidoras()` que obtém a lista de distribuidoras e a função `seleciona_distribuidora()`, que, dada uma distribuidora, irá realizar a sua seleção.

In [16]:
def gera_lista_distribuidoras():
    try:
        select_distribuidora = Select(driver.find_element_by_xpath("//select[@id='select_agente']"))
        select_distribuidora_opcoes = select_distribuidora.options
        lista_nomes = [opcao.get_attribute("value") for opcao in select_distribuidora_opcoes]
        return lista_nomes
    except Exception as e:
        erro_stale_element('select_agente')
        return None
    
def seleciona_distribuidora(distribuidora):
    try:
        select_distribuidora = Select(driver.find_element_by_xpath("//select[@id='select_agente']"))
        select_distribuidora.select_by_value(distribuidora)
        selecionou = verifica_selecao(select_distribuidora, distribuidora)
        while selecionou != True:
            selecionou = verifica_selecao(select_distribuidora, distribuidora)
        return selecionou
    except Exception as e:
        print(e)
        if e == StaleElementReferenceException:
            erro_stale_element('select_agente')
            seleciona_distribuidora(distribuidora)

De forma semelhante, vamos definir funções que geram listas de anos e meses e selecionam o ano e o mês indicado.

In [17]:
def gera_lista_anos():
    try:
        select_ano = Select(driver.find_element_by_xpath("//select[@id='select_ano']"))
        select_ano_opcoes = select_ano.options
        lista_anos = [int(opcao.get_attribute("value")) for opcao in select_ano_opcoes]
        return lista_anos
    except Exception as e:
        erro_stale_element('select_ano')
        return None


def seleciona_ano(ano):
    try:
        select_ano = Select(driver.find_element_by_xpath("//select[@id='select_ano']"))
        select_ano.select_by_value(ano)
        selecionou = verifica_selecao(select_ano, ano)
        while selecionou != True:
            selecionou = verifica_selecao(select_ano, ano)
        return selecionou
    except Exception as e:
        erro_stale_element('select_ano')
        seleciona_ano(ano)

def gera_lista_meses():
    try:
        select_mes = Select(driver.find_element_by_xpath("//select[@id='select_mes']"))
        select_mes_opcoes = select_mes.options
        lista_meses = [int(opcao.get_attribute("value")) for opcao in select_mes_opcoes]
        return lista_meses
    except Exception as e:
        erro_stale_element('select_mes')
        return None

def seleciona_mes(mes):
    try:
        select_mes = Select(driver.find_element_by_xpath("//select[@id='select_mes']"))
        select_mes.select_by_value(mes)
        selecionou = verifica_selecao(select_mes, mes)
        while selecionou != True:
            selecionou = verifica_selecao(select_mes, mes)
        return selecionou
    except Exception as e:
        erro_stale_element('select_mes')
        return None

Agora vamos definir uma função que armazena os dados da tabela gerada na página da ANEEL em um dataframe.

In [18]:
import sys
import os
def armazena_dados_df(distribuidora, ano, mes):
    try:
        indice = 0 if df_novos_dados.empty else df_novos_dados.index.max() + 1

        tabela_pagina = driver.find_element_by_xpath("//div[@id='div_relatorio']")
        linhas_tabela_pagina = tabela_pagina.find_element_by_tag_name("table").find_elements_by_tag_name("tr")
        count_linhas_armazenadas = 0
        for linha in linhas_tabela_pagina:
            if linha.get_attribute("class") == 'tr-cabecalho-tabela':
                for cabecalho in linha.find_elements_by_tag_name("td"):
                    if cabecalho.get_attribute("colspan") == "7":
                        if cabecalho.text != distribuidora + ' - ' + ano + ' - ' + dict_meses[mes]:
                            return False

            if linha.get_attribute("class") in ["odd", "even"]:
                colunas_linha = linha.find_elements_by_tag_name("td")
                dados_armazenados = [distribuidora, ano, mes]
                for coluna in colunas_linha:
                    dado = coluna.text.replace(".", "").replace(",", ".")
                    if dado == "TOTAL":
                        break
                    else:
                        try:
                            dados_armazenados.append(int(dado))
                        except:
                            try:
                                dados_armazenados.append(float(dado))
                            except:
                                dados_armazenados.append(dado)

                # Adicionando linha de dados à tabela
                if len(dados_armazenados) > 3:
                    df_novos_dados.loc[indice] = dados_armazenados
                    count_linhas_armazenadas += 1
                    indice += 1
                    if count_linhas_armazenadas == 15:
                        return True

    except Exception as e:
        exc_type, exc_obj, exc_tb = sys.exc_info()
        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        erro_stale_element('div_relatorio')
        armazena_dados_df(distribuidora, ano, mes)

Uma vez que os dados estejam armazenados em um dataframe, precisamos transferi-los para o nosso banco de dados. Vamos usar a função `sql_connection()` para nos conectar ao banco, a função `query_to_df()` para fazer uma query em nosso banco de dados existente e a função `insert_data()` para inserir os novos dados coletados. Não esqueça de inserir a senha de acesso ao banco de dados.

In [19]:
import psycopg2
import pandas.io.sql as sqlio

def sql_connection():
    # Realiza a conexão com o banco de dados PostgreSQL e retorna a variável que será utilizada para interagir com o banco
    database = 'idec-reclamacoes'
    user = 'postgres'
    password = 'idec2021'
    host = '34.72.240.61'
    port = '5432'
    con = psycopg2.connect(database=database,
                           user=user,
                           password=password,
                           host=host,
                           port=port)
    return con

def query_to_df(query):
    # Dada uma query PostgreSQL, realiza a query no banco de dados e armazena o resultado em um dataframe
    con = sql_connection()
    data_set = sqlio.read_sql_query(query, con)
    return data_set

def insert_data(data_set):
    # Alimenta a tabela PostgreSQL com os dados dos CNPJ's
    con = sql_connection()
    meses_inseridos = []
    with con.cursor() as cur:
        for index, row in data_set.iterrows():
            if df_existente[(df_existente['distribuidora']==row.distribuidora) & (df_existente['ano']==int(row.ano)) & (df_existente['mes']==int(row.mes))].empty == True:
                query_insert = "insert into reclamacoes_aneel values ('" + row.distribuidora + "', " + ', '.join([row.ano,row.mes])+ ", '" + row.descricao + "', " + ', '.join([str(x) for x in [row.reclamacoes_recebidas, row.reclamacoes_procedentes, row.prazo_medio_encerramento_horas, row.prazo_medio_encerramento_dias, row.reclamacoes_improcedentes, row.reclamacoes_totais]])+")"
                cur.execute(query_insert)
                meses_inseridos.append(int(row.mes))
        con.commit()
        return meses_inseridos
    

Agora que todas as funções já estão definidas, podemos partir pra ação e usar o script abaixo pra coletar os dados no site da ANEEL e armazená-los em nosso banco de dados.

In [24]:
df_existente = query_to_df('select * from reclamacoes_aneel')

driver = requisicao_html()
time.sleep(1)
if primeiros_selects():
    lista_distribuidoras = gera_lista_distribuidoras()
    while lista_distribuidoras == None: lista_distribuidoras = gera_lista_distribuidoras()
    lista_distribuidoras_unicos = sorted([elemento for elemento in set(lista_distribuidoras)])
    
    for distribuidora in lista_distribuidoras_unicos[99:]:
        df_novos_dados = clean_df_novos_dados()
        selecionou_distribuidora = seleciona_distribuidora(distribuidora)
        while selecionou_distribuidora != True: selecionou_distribuidora = seleciona_distribuidora(distribuidora)
        lista_anos = gera_lista_anos()
        while lista_anos == None: lista_anos = gera_lista_anos()         

        for ano in [str(x) for x in list(set(lista_anos) & set(anos_selecionados))]:
            selecionou_ano = seleciona_ano(ano)
            while selecionou_ano != True: selecionou_ano = seleciona_ano(ano)
            lista_meses = gera_lista_meses()
            while lista_meses == None: lista_meses = gera_lista_meses()

            for mes in [str(x) for x in list(set(lista_meses) & set(meses_selecionados))]:
                selecionou_mes = seleciona_mes(str(mes))
                while selecionou_mes != True: selecionou_mes = seleciona_mes(str(mes))                           
                armazenou_dados_df = armazena_dados_df(distribuidora, ano, mes)
                while armazenou_dados_df != True: armazenou_dados_df = armazena_dados_df(distribuidora, ano, mes)
        
        df_novos_dados = df_novos_dados.drop_duplicates()
        dados_inseridos = insert_data(df_novos_dados)
        if dados_inseridos != None:
            print(lista_distribuidoras_unicos.index(distribuidora), distribuidora, "inseridos dados dos meses:", set(dados_inseridos))
        
                

99 Iguaçu Energia inseridos dados dos meses: set()
100 João Cesa inseridos dados dos meses: {1}
101 Light inseridos dados dos meses: set()
102 Mux Energia inseridos dados dos meses: {1}
103 Nacional inseridos dados dos meses: set()
104 Nova Palma inseridos dados dos meses: set()
105 RGE inseridos dados dos meses: set()
106 RGE Sul inseridos dados dos meses: {1, 2}
107 Roraima Energia inseridos dados dos meses: set()
108 Santa Maria inseridos dados dos meses: set()
109 Sulgipe inseridos dados dos meses: set()
110 Vale Paranapanema inseridos dados dos meses: set()


Por fim, vamos fazer algumas alterações no banco de dados que nos ajudarão a trabalhar no dashboard.

In [30]:
con = sql_connection()
with con.cursor() as cur:
    cur.execute("update reclamacoes_aneel set data_ref = to_date(concat('01/', mes, '/', ano), 'DD/MM/YYYY')")
    cur.execute("update reclamacoes_aneel as ra set uf = du.uf from distribuidoras_uf du where ra.distribuidora = du.distribuidora")
    cur.execute("update distribuidoras_ranking as dr set reclamacoes_2020 = sub.reclamacoes_2020 from (select distribuidora, sum(reclamacoes_recebidas) as reclamacoes_2020 from reclamacoes_aneel ra  where ano = 2020 group by distribuidora) as sub where dr.distribuidora = sub.distribuidora")
    cur.execute("update distribuidoras_ranking as dr set indicador = 100000*(reclamacoes_2020/consumidores::float)")
    cur.execute("update distribuidoras_ranking as dr set ranking = rank_number from (select *, rank() over (order by indicador desc) rank_number from distribuidoras_ranking) as sub where sub.distribuidora = dr.distribuidora")
con.commit()